# **Import all required libraries**

In [1]:
import pickle
import gzip
import os
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import mixed_precision
# import tensorflow_addons as tfa
# import keras_tuner
import cv2
import numpy as np
import re
import os
import torch
from tqdm.notebook import tqdm
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.efficientnet import EfficientNetB7, EfficientNetB0 
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import mixed_precision
from openpyxl import Workbook, load_workbook
import platform
from progress.bar import Bar
import tqdm
import multiprocessing
import concurrent.futures

# **Enable fp16**

In [2]:
tf.config.list_physical_devices('GPU')
tf.config.run_functions_eagerly(True)
# Set the number of threads as desired
num_threads = 32

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver() #TPU detection
# except ValueError:
#     tpu = None

# if tpu:
#     policyConfig ='mixed_bfloat16'
# else:
#     policyConfig ='mixed_float16'

# policy = tf.keras.mixed_precision.Policy(policyConfig)
# tf.keras.mixed_precision.set_global_policy(policy)

In [4]:
mixed_precision.set_global_policy('float16')

# **Mention the destination of the files to access:**

In [5]:

# validation
vw_dest = "Dataset/excels/ISH_News_Business VAL.xlsx"
vo_dest = "/Dataset/Pickles/excel_dataVal.pickle"
vf_dest = "Dataset/Final folder for frames"

# test
tw_dest = "Dataset/excels/ISH_News_Business TEST.xlsx"
to_dest = "/Dataset/Pickles/excel_dataTest.pickle"
tf_dest = "Dataset/Final folder for frames"

# train
w_dest = "Dataset/excels/ISH_News_Business TRAIN.xlsx"
o_dest = "/Dataset/Pickles/excel_data.pickle"
f_dest = "Dataset/Final folder for frames"

In [6]:
def cls():
    system = platform.system()
    if system == 'Windows':
        os.system('cls')
    elif system == 'Linux':
        os.system('clear')
    elif system == 'MacOS':
        os.system('clear')

# **Initialise the CNN model here:**

In [7]:
base_model = EfficientNetB7(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
feature_extractor = Model(inputs=base_model.input, outputs=x)

# **Function for extraction of features:**

In [8]:

def get_features(filename, destination):
    input_string = filename
    pattern = r'\d+'
    match = re.search(pattern, input_string)
    if match:
        first_match = match.group()
        # print(first_match)
        # Get file path name
        input_folder = os.getcwd()+"/"+destination+"/"+first_match+"/"+input_string
        file_paths_frames = [file for file in sorted(os.listdir(input_folder)) if file.endswith(".jpg")]

        # base_model needs to be initialised first
        # x = base_model.output
        # x = GlobalAveragePooling2D()(x)
        # feature_extractor = Model(inputs=base_model.input, outputs=x)
    
    
        features_listofList=[]
        for indx, frame_file in enumerate(file_paths_frames):
            # Get the file
            frame_filename =  input_folder+"/" + frame_file
            print("frame_filename============", frame_filename,"\n")
            image = cv2.imread(frame_filename)
                
            # Image processing part
        
            # Define the coordinates of the ROI (top-left and bottom-right)
            x1, y1 = 535, 0  # Top-left corner of ROI
            x2, y2 = 1385, 1080  # Bottom-right corner of ROI
            #210*260
            # Crop the image to the specified ROI
            image = image[y1:y2, x1:x2]
            image = cv2.resize(image, (224, 224))  # Resize to the input size of MobileNet

            # Preprocess the image
            image = preprocess_input(image)
        
            # Expand dimensions to match the expected input shape (batch size of 1)
            image = np.expand_dims(image, axis=0)
                
            # Extract spatial embedding
            spatial_embedding = feature_extractor.predict(image)[0]
                
            features_listofList.append(spatial_embedding)
    else:
        print("No match found.")
        return None

    # after this, the video must be in the form of features
    return torch.tensor(np.array(features_listofList))
    #return torch.tensor(features_listofList)

# **Function to create the pickle file:**

In [9]:
def create_pickle(workbook_dest, output_dest, frame_dest):
    # Load the excel file
    workbook = load_workbook(workbook_dest)
    sheet = workbook.active

    # Extract data from the Excel file
    excel_data = []
    for row in sheet.iter_rows(values_only=True):
        excel_data.append(row)
        # print(excel_data,"\n")
        # print(excel_data)list_of_inputs = []
    
    # Get the features
    list_of_inputs = []
    
    def process_data(tmp, frame_dest):
        features = get_features(str(tmp[0]), frame_dest)
        if features is not None:
            data_dict = {
                'name': tmp[0],
                'signer': tmp[1],
                'gloss': tmp[2],
                'text': tmp[3],
                'sign': features + 1e-8
            }
            return data_dict
        else:
            return None

    # Assuming excel_data is a list of tuples
    # excel_data = [...]

    # # Set the number of threads as desired
    # num_threads = 4

    list_of_inputs = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Use the executor to map the process_data function to each element in excel_data
        results = list(executor.map(lambda tmp: process_data(tmp, frame_dest), excel_data))

    # Filter out None values (where features is None)
    list_of_inputs = [data_dict for data_dict in results if data_dict is not None]

    # Now list_of_inputs contains the processed data from the loop in parallel

    # print("\nlist_of_input:\n")
    # print(list_of_inputs)
    with gzip.open(os.getcwd() + output_dest,'wb') as f:
        pickle.dump(list_of_inputs,f)

In [10]:
# def create_pickle(workbook_dest, output_dest, frame_dest):
#     # Load the excel file
#     workbook = load_workbook(workbook_dest)
#     sheet = workbook.active

#     # Extract data from the Excel file
#     excel_data = []
#     for row in sheet.iter_rows(values_only=True):
#         excel_data.append(row)
#         # print(excel_data,"\n")
#         # print(excel_data)list_of_inputs = []
    
#     # Get the features
#     list_of_inputs = []
    
#     for tmp in excel_data:
#         features = get_features(str(tmp[0]),frame_dest)
#         if(features!= None):
#             data_dict = {}
#             data_dict['name'] = tmp[0]
#             data_dict['signer'] = tmp[1]
#             data_dict['gloss'] = tmp[2]
#             data_dict['text'] = tmp[3]
#             data_dict['sign'] = features + 1e-8
#             #print(data_dict)
#             #input()
#             list_of_inputs.append(data_dict)

#     # print("\nlist_of_input:\n")
#     # print(list_of_inputs)
#     with gzip.open(os.getcwd() + output_dest,'wb') as f:
#         pickle.dump(list_of_inputs,f)

# **Call for training:**

In [11]:
# Testing pickle
create_pickle(tw_dest, to_dest, tf_dest)

print("Testing pickle Creation Successful")

frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for frames/1/1_67.00/frame_1943.jpg 

frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for frames/1/1_190.00/frame_5511.jpg 

frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for frames/1/1_90.00/frame_2611.jpg 

frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for frames/1/1_195.00/frame_5655.jpg 

1/1 [==============================] - 8s 8s/step
frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for frames/1/1_195.00/frame_5657.jpg 

1/1 [==============================] - 0s 90ms/step
frame_filename============ c:\Users\Rahul Gogoi\Files\islt_directml\Pre-Processing\Pickle maker/Dataset/Final folder for

InvalidArgumentError: Graph execution error:

Detected at node 'model/block4g_se_reshape/strided_slice' defined at (most recent call last):
    File "c:\Users\Rahul Gogoi\.conda\envs\slt_directml\lib\threading.py", line 973, in _bootstrap
      self._bootstrap_inner()
    File "c:\Users\Rahul Gogoi\.conda\envs\slt_directml\lib\threading.py", line 1016, in _bootstrap_inner
      self.run()
    File "c:\Users\Rahul Gogoi\.conda\envs\slt_directml\lib\threading.py", line 953, in run
      self._target(*self._args, **self._kwargs)
    File "c:\Users\Rahul Gogoi\.conda\envs\slt_directml\lib\concurrent\futures\thread.py", line 83, in _worker
      work_item.run()
    File "c:\Users\Rahul Gogoi\.conda\envs\slt_directml\lib\concurrent\futures\thread.py", line 58, in run
      result = self.fn(*self.args, **self.kwargs)
    File "C:\Users\Rahul Gogoi\AppData\Local\Temp\ipykernel_41584\2673706345.py", line 40, in <lambda>
      results = list(executor.map(lambda tmp: process_data(tmp, frame_dest)

In [ ]:
# Validation pickle
create_pickle(vw_dest, vo_dest, vf_dest)

print("Validation/dev pickle Creation Successful")

In [ ]:
# Training pickle
create_pickle(w_dest, o_dest, f_dest)

print("Training pickle Creation Successful")